In [1]:
!ls

gold_recovery_full_new.csv  gold_recovery_train_new.csv
gold_recovery_test_new.csv  ntb.ipynb


In [2]:
import pandas as pd 

In [4]:
data = pd.read_csv('gold_recovery_train_new.csv')

In [9]:
len(data.columns)

87